# Infera Agent Demo

This notebook demonstrates how to use Infera as a tool for AI agents.

**What you'll learn:**
1. Using the Python SDK to interact with Infera
2. Understanding risk scores with explanations
3. Semantic search across risk factors
4. Integrating with LangGraph agents

---


## Setup

First, ensure the Infera API is running:
```bash
cd backend && uvicorn api.main:app --reload
```


In [ ]:
import sys
sys.path.insert(0, '..')

from sdk.infera_client import InferaClient

# Initialize client
client = InferaClient(base_url="http://localhost:8000")

# Check health
health = client.health()
print(f"API Status: {health['status']}")


## 1. List Available Filings

See what companies have been analyzed.


In [ ]:
filings = client.list_filings()

print(f"Found {len(filings)} filings:\n")
for f in filings:
    summary_status = "✅" if f.has_summary else "❌"
    print(f"  {f.ticker} ({f.filing_type}) - Summary: {summary_status}")


## 2. Semantic Search

Find risks related to a specific topic using natural language.


In [ ]:
# Search for cybersecurity risks
results = client.search("cybersecurity data breach unauthorized access", limit=5)

print(f"Found {len(results)} results:\n")
for i, r in enumerate(results, 1):
    print(f"{i}. [{r.ticker}] Score: {r.score:.3f}")
    print(f"   {r.text[:150]}...")
    print()


## 3. Explain a Risk Score

Understand *why* a paragraph scored high or low using token attribution.


In [ ]:
# Pick a paragraph to explain
if results:
    paragraph_id = results[0].paragraph_id
    
    explanation = client.explain(paragraph_id, top_n=10)
    
    print(f"Paragraph ID: {explanation.paragraph_id}")
    print(f"Risk Score: {explanation.score:.3f}")
    print(f"Confidence: {explanation.confidence:.1%}")
    print(f"Category: {explanation.risk_category} ({explanation.category_confidence:.1%})")
    print()
    print("Top contributing tokens:")
    for token in explanation.top_tokens[:5]:
        print(f"  • '{token.token}': +{token.contribution:.4f}")


## 4. Get Raw Embeddings

For advanced use cases, retrieve the actual embedding vectors.


In [ ]:
if results:
    paragraph = client.get_paragraph(results[0].paragraph_id, include_embedding=True)
    
    print(f"Paragraph ID: {paragraph.paragraph_id}")
    print(f"Score: {paragraph.score:.3f}")
    print(f"Confidence: {paragraph.confidence:.1%}")
    
    if paragraph.embedding:
        print(f"\nEmbedding shape: {len(paragraph.embedding)} dimensions")
        print(f"First 5 values: {paragraph.embedding[:5]}")


---

## 5. LangGraph Integration

Use Infera as a tool in a LangGraph agent.


In [ ]:
# Import LangGraph tools
from examples.langgraph_tool import (
    analyze_risks,
    explain_risk_score,
    search_risks,
    compare_companies,
    get_filing_summary,
)

# Test search tool directly
result = search_risks.invoke({"query": "supply chain disruption", "limit": 3})
print(result)


In [ ]:
# Create a full agent (requires OPENAI_API_KEY)
import os

if os.getenv("OPENAI_API_KEY"):
    from examples.langgraph_tool import create_infera_agent
    from langchain_core.messages import HumanMessage
    
    agent = create_infera_agent()
    
    # Ask a question
    response = agent.invoke({
        "messages": [HumanMessage(content="What are the main cybersecurity risks disclosed by tech companies?")]
    })
    
    print(response["messages"][-1].content)
else:
    print("Set OPENAI_API_KEY to run the agent example")


---

## 6. OpenAI Function Calling

Use Infera with OpenAI's function calling API.


In [ ]:
from examples.openai_functions import INFERA_FUNCTIONS, execute_function
import json

# Show available functions
print("Available functions:")
for f in INFERA_FUNCTIONS:
    print(f"  • {f['function']['name']}")

# Execute a function directly
result = execute_function(
    "search_risks",
    json.dumps({"query": "regulatory compliance", "limit": 3})
)
print("\nSearch result:")
print(json.dumps(result, indent=2))


---

## Summary

This notebook demonstrated:

1. **SDK Usage** - Clean Python interface to Infera API
2. **Semantic Search** - Find risks by meaning, not keywords
3. **Explainability** - Token-level attribution for every score
4. **Confidence Scores** - Percentile-based uncertainty quantification
5. **Raw Embeddings** - Access FinBERT vectors for custom analysis
6. **LangGraph Tools** - Ready-to-use tools for LangGraph agents
7. **OpenAI Functions** - Schema for Assistants API integration


In [ ]:
# Cleanup
client.close()
print("✅ Demo complete!")
